# Covid-19 Hotspots
> Insights into the Covid-19 hotspots in Ontario, Canada.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/test.png

In [1]:
#hide
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import altair as alt

from IPython.display import HTML, display
import pytz

import ipywidgets as widgets

from datetime import datetime, timedelta, timezone

print('Libraries successfully imported.')

Libraries successfully imported.


In [2]:
#hide
#%%time

url = 'https://data.ontario.ca/dataset/f4112442-bdc8-45d2-be3c-12efae72fb27/resource/455fd63b-603d-4608-8216-7d8647f43350/download/conposcovidloc.csv'
url_2 = 'https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/ed270bb8-340b-41f9-a7c6-e8ef587e6d11/download/covidtesting.csv'
url_3 = 'https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/8a88fe6d-d8fb-41a3-9d04-f0550a44999f/download/daily_change_in_cases_by_phu.csv'

#geo = gpd.read_file('https://data.ontario.ca/dataset/f4112442-bdc8-45d2-be3c-12efae72fb27/resource/4f39b02b-47fe-4e66-95b6-e6da879c6910/download/conposcovidloc.geojson') 

src_conpos = pd.read_csv(url, index_col=0, parse_dates=['Accurate_Episode_Date', 'Case_Reported_Date', 'Test_Reported_Date', 'Specimen_Date']).reset_index()
src_testing = pd.read_csv(url_2, index_col=0, parse_dates=['Reported Date']).reset_index()
src_daily = pd.read_csv(url_3, index_col=0, parse_dates=['Date']).reset_index()

# duplicate entry for December 5th, 2020 causing error
src_testing.drop_duplicates(inplace=True)

refreshtime = datetime.now()
cached=False

print('Source files successfully loaded.')

Source files successfully loaded.


In [3]:
#hide
last_date = src_testing['Reported Date'].max()
init_date = src_conpos['Accurate_Episode_Date'].min()
display(HTML("Ontario data set last updated on: " + last_date.strftime("%x")))

In [4]:
#hide
src_daily.head()

,Date,Algoma_Public_Health_Unit,Brant_County_Health_Unit,Chatham-Kent_Health_Unit,Durham_Region_Health_Department,Eastern_Ontario_Health_Unit,Grey_Bruce_Health_Unit,Haldimand-Norfolk_Health_Unit,"Haliburton,_Kawartha,_Pine_Ridge_District_Health_Unit",Halton_Region_Health_Department,...,Simcoe_Muskoka_District_Health_Unit,Southwestern_Public_Health,Sudbury_&_District_Health_Unit,Thunder_Bay_District_Health_Unit,Timiskaming_Health_Unit,Toronto_Public_Health,Wellington-Dufferin-Guelph_Public_Health,Windsor-Essex_County_Health_Unit,York_Region_Public_Health_Services,Total
0,2020-03-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2020-03-25,0.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,17.0,1.0,1.0,5.0,46
2,2020-03-26,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,...,1.0,0.0,0.0,0.0,1.0,21.0,1.0,2.0,5.0,69
3,2020-03-27,0.0,0.0,0.0,5.0,0.0,1.0,0.0,14.0,1.0,...,4.0,2.0,1.0,0.0,0.0,22.0,0.0,0.0,34.0,124
4,2020-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
#hide
df_phu = src_daily.drop(columns=['Total']).melt(id_vars='Date', value_name='Confirmed') \
.rename(columns={"variable" : "Public_Health_Unit"}) \
.fillna(0).sort_values(['Date']).reset_index(drop=True)

df_phu.head()

,Date,Public_Health_Unit,Confirmed
0,2020-03-24,Algoma_Public_Health_Unit,0.0
1,2020-03-24,Middlesex-London_Health_Unit,0.0
2,2020-03-24,Renfrew_County_and_District_Health_Unit,0.0
3,2020-03-24,"Region_of_Waterloo,_Public_Health",0.0
4,2020-03-24,North_Bay_Parry_Sound_District_Health_Unit,0.0


In [6]:
#hide
df_phu[df_phu['Date'] == df_phu['Date'].max()].sort_values(['Public_Health_Unit']).sort_values('Confirmed', ascending=False).reset_index(drop=True).head(10)

,Date,Public_Health_Unit,Confirmed
0,2020-12-19,Toronto_Public_Health,665.0
1,2020-12-19,Peel_Public_Health,448.0
2,2020-12-19,York_Region_Public_Health_Services,174.0
3,2020-12-19,Windsor-Essex_County_Health_Unit,170.0
4,2020-12-19,Hamilton_Public_Health_Services,118.0
5,2020-12-19,Niagara_Region_Public_Health_Department,118.0
6,2020-12-19,Durham_Region_Health_Department,104.0
7,2020-12-19,Middlesex-London_Health_Unit,93.0
8,2020-12-19,"Region_of_Waterloo,_Public_Health",83.0
9,2020-12-19,Halton_Region_Health_Department,72.0


In [7]:
#hide
import warnings
warnings.filterwarnings("ignore")

ontario = df_phu.groupby(['Date'])['Confirmed'].sum().reset_index()
toronto = df_phu[df_phu['Public_Health_Unit'].isin(['Toronto_Public_Health'])]
peel = df_phu[df_phu['Public_Health_Unit'].isin(['Peel_Public_Health'])]
york = df_phu[df_phu['Public_Health_Unit'].isin(['York_Region_Public_Health_Services'])]
halton = df_phu[df_phu['Public_Health_Unit'].isin(['Halton_Region_Health_Department'])]

toronto['Rolling 7d'] = toronto['Confirmed'].rolling(7).mean()
peel['Rolling 7d'] = peel['Confirmed'].rolling(7).mean()
york['Rolling 7d'] = york['Confirmed'].rolling(7).mean()
halton['Rolling 7d'] = halton['Confirmed'].rolling(7).mean()
ontario['Rolling 7d'] = ontario['Confirmed'].rolling(7).mean()

In [8]:
#hide
fig_hot = px.line(width=1000, template='seaborn'
                  )

fig_hot.add_trace(go.Scatter(
    x=ontario['Date'],
    y=ontario['Rolling 7d'],
    mode='lines',
    name='Ontario Overall',
    line=dict(color="black", dash="dot")
    ))
fig_hot.add_trace(go.Scatter(
    x=toronto['Date'],
    y=toronto['Rolling 7d'],
    mode='lines',
    name='Toronto',
    #line=dict(color="yellow", dash="solid")
    ))

fig_hot.add_trace(go.Scatter(
    x=peel['Date'],
    y=peel['Rolling 7d'],
    mode='lines',
    name='Peel',
    #line=dict(color="red", dash="solid")
    ))
fig_hot.add_trace(go.Scatter(
    x=halton['Date'],
    y=halton['Rolling 7d'],
    mode='lines',
    name='Halton',
    #line=dict(color="orange", dash="solid")
    ))
fig_hot.add_trace(go.Scatter(
    x=york['Date'],
    y=york['Rolling 7d'],
    mode='lines',
    name='York',
    #line=dict(color="green", dash="solid")
    ))

fig_hot.update_layout(title='Confirmed Cases for Ontario Covid-19 Hotspots',
                      xaxis_title='Date',
                      yaxis_title='Cases - 7d Rolling Avg')